# Compare Subset Selection Methods
Compare discprency based subselection to easiest/hardest tasks (determined via the Pareto front)

## Prerequisite
Full data for each scenario needed and the results from the subselection procedure.

## Limitations
Using the subset size of the discrepancy subset.

## Define Scenario


In [ ]:
from __future__ import annotations

scenario = "blackbox"
set_id = "full"
subset_size = 30
paths = ["../runs/SMAC3-BlackBoxFacade", "../runs/RandomSearch", "../runs/Nevergrad-CMA-ES"]
performance_fn = "/scratch/hpc-prf-intexml/cbenjamins/repos/CARP-S-Experiments/lib/CARP-S/subselection/data/BB/default/df_crit.csv"

# scenario = "multi-fidelity-objective"
# set_id = "full"
# subset_size = 9
# paths = ["../runs_MOMF/SMAC3-MOMF-GP", "../runs_MOMF/RandomSearch", "../runs_MOMF/Nevergrad-DE"]
# performance_fn = "/scratch/hpc-prf-intexml/cbenjamins/repos/CARP-S-Experiments/lib/CARP-S/subselection/data/MOMF/lognorm/df_crit.csv"

# scenario = "multi-objective"
# set_id = "full"
# subset_size = 10
# paths = ["../runs_MO/Optuna-MO", "../runs_MO/RandomSearch", "../runs_MO/Nevergrad-DE"]
# performance_fn = "/scratch/hpc-prf-intexml/cbenjamins/repos/CARP-S-Experiments/lib/CARP-S/subselection/data/MO/lognorm/df_crit.csv"

# scenario = "multi-fidelity"
# set_id = "full"
# subset_size = 20
# paths = ["../runs/SMAC3-Hyperband", "../runs/SMAC3-MultiFidelityFacade", "../runs_/DEHB"]

## Load data

In [ ]:
import logging
import sys
from pathlib import Path

import pandas as pd
from carps.analysis.gather_data import normalize_logs
from carps.analysis.run_autorank import get_df_crit
from carps.analysis.utils import filter_only_final_performance

logging.disable(sys.maxsize)

original_optimizers = {
    "blackbox": ["RandomSearch", "SMAC3-BlackBoxFacade", "Nevergrad-CMA-ES"],
    "multi-objective": ["RandomSearch", "Optuna-MO", "Nevergrad-DE"],
    "multi-fidelity": ["SMAC3-Hyperband", "SMAC3-MultiFidelityFacade", "DEHB"],
    "multi-fidelity-objective": ["RandomSearch", "SMAC3-MOMF-GP", "Nevergrad-DE"],
}

def load_set(paths: list[str], set_id: str = "unknown") -> tuple[pd.DataFrame, pd.DataFrame]:
    logs = []
    for p in paths:
        fn = Path(p) / "trajectory.parquet"
        if not fn.is_file():
            fn = Path(p) / "logs.parquet"
        logs.append(pd.read_parquet(fn))

    df = pd.concat(logs).reset_index(drop=True)
    df_cfg = pd.concat([pd.read_parquet(Path(p) / "logs_cfg.parquet") for p in paths]).reset_index(drop=True)
    df["set"] = set_id
    return df, df_cfg

fn = f"../data/{scenario}_{set_id}_logs.parquet"
if Path(fn).is_file():
    df = pd.read_parquet(fn)
else:
    D = []
    for rundir in paths:
        df, df_cfg = load_set([rundir], set_id="full")
        D.append(df)

    df = pd.concat(D).reset_index(drop=True)
    del D

    df = normalize_logs(df)

    normalize_performance = False
    perf_col = "trial_value__cost_inc_norm" if normalize_performance else "trial_value__cost_inc"

    df.to_parquet(f"../data/{scenario}_{set_id}_logs.parquet")
df = filter_only_final_performance(df=df)
df_crit = get_df_crit(df)

## Generate subsets

1. Random I: randomly sample from full set of tasks
2. Random II: randomly sample from full set of tasks, but keep fraction per benchmark the same
3. Easiest: Find easiest tasks with non-dominated sorting, select top-k
4. Hardest: Find hardest tasks with non-dominated sorting, select top-k

For all of them: Divide them into dev and test.
For the random subsets: Create n splits.
As this is stochastic, repeat the procedure for 5 seeds.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from carps.utils.pareto_front import pareto
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit


def read_set(fn: Path) -> pd.DataFrame:
    df = pd.read_csv(fn)
    df["task_id"] = df["task_id"].apply(lambda x: "bbob/" + x if x.startswith("noiseless") else x)
    df["benchmark_id"] = df["task_id"].apply(lambda x: x.split("/")[0])
    return df.melt(id_vars=["task_id", "benchmark_id"], value_vars=original_optimizers[scenario], var_name="optimizer_id", value_name="performance")

task_ids = df["task_id"].unique()
# print(task_ids)
benchmark_ids = [pid.split("/")[0] for pid in task_ids]
n_tasks = len(task_ids)

performance_fn = Path(performance_fn)
path_subset_dev = performance_fn.parent / f"subset_{subset_size}.csv"
subset_dev = read_set(path_subset_dev)

path_subset_test = performance_fn.parent / f"subset_complement_subset_{subset_size}.csv"
subset_test = read_set(path_subset_test)

task_ids_dev = subset_dev["task_id"].to_list()
task_ids_test = subset_test["task_id"].to_list()

print(n_tasks)



seeds = np.arange(0, 5)

split_classes = [StratifiedShuffleSplit, ShuffleSplit]

def get_set_type(x: str) -> str:
    if x.startswith("class"):
        _id = "StratifiedShuffleSplit" if "StratifiedShuffleSplit" in x else "ShuffleSplit"
        if x.endswith("dev"):
            return f"{_id}_dev"
        return f"{_id}_test"
    if x.startswith("pareto"):
        _id = "pareto"
        if x.endswith("dev"):
            return f"{_id}_dev"
        return f"{_id}_test"
    return x

new_subset_performance = []



def find_pareto(df_crit: pd.DataFrame, which: str = "easiest") -> pd.DataFrame:
    n_tries = 1000
    counter = 0
    index = []
    while len(df_crit) > 0:
        crit_points = df_crit.values
        sign = 1 if which == "easiest" else -1
        ids_pareto = pareto(sign * crit_points)
        index.extend(df_crit[ids_pareto].index)
        df_crit = df_crit[~ids_pareto]

        if counter > n_tries:
            break
        counter += 1
    return index

# Pareto
print("Pareto")
df_crit_ = get_df_crit(df[df["task_id"].isin(task_ids)], perf_col="trial_value__cost_inc_norm")
task_ids_easiest = find_pareto(df_crit_, which="easiest")[:int(subset_size * 2)]
task_ids_hardest = find_pareto(df_crit_, which="hardest")[:int(subset_size * 2)]

split_class = ShuffleSplit
for set_origin, pids in zip(["pareto_easiest", "pareto_hardest"], [task_ids_easiest, task_ids_hardest], strict=False):
    bids = [pid.split("/")[0] for pid in pids]
    X = pids
    y = bids
    n_splits = len(pids) // (subset_size * 2)

    for seed in seeds:
        sss = split_class(n_splits=n_splits, train_size=subset_size/len(pids), test_size=subset_size/len(pids), random_state=seed)
        sss.get_n_splits(X, y)
        for i, (train_index, test_index) in enumerate(sss.split(X, y)):


            assert len(train_index) == subset_size

            ids_dev = task_ids[train_index]
            ids_test = task_ids[test_index]
            for _set_id, ids in zip(["dev", "test"], (ids_dev, ids_test), strict=False):
                new_df = df[df["task_id"].isin(ids)].copy()
                set_id = f"{set_origin}_class_{split_class.__name__}_split_{i}_seed_{seed}_subset_{_set_id}"
                new_df["set"] = set_id
                print(set_id, len(train_index), len(test_index), new_df["task_id"].nunique())
                new_subset_performance.append(new_df)


print("Standard")
for set_id, ids in zip(["full", "discrepancy_subset_dev", "discrepancy_subset_test"], (task_ids, task_ids_dev, task_ids_test), strict=False):
    new_df = df[df["task_id"].isin(ids)].copy()
    new_df["set"] = set_id
    new_subset_performance.append(new_df)

# Random Splits
print("Random")
n_splits = n_tasks // (subset_size * 2)
# n_splits = 1
print(n_splits)
X = task_ids
y = benchmark_ids
for split_class in split_classes:
    for seed in seeds:
        sss = split_class(n_splits=n_splits, train_size=subset_size/n_tasks, test_size=subset_size/n_tasks, random_state=seed)
        sss.get_n_splits(X, y)
        for i, (train_index, test_index) in enumerate(sss.split(X, y)):


            assert len(train_index) == subset_size

            ids_dev = task_ids[train_index]
            ids_test = task_ids[test_index]
            for _set_id, ids in zip(["dev", "test"], (ids_dev, ids_test), strict=False):
                new_df = df[df["task_id"].isin(ids)].copy()
                set_id = f"class_{split_class.__name__}_split_{i}_seed_{seed}_subset_{_set_id}"
                new_df["set"] = set_id
                print(set_id, len(train_index), len(test_index), new_df["task_id"].nunique())
                new_subset_performance.append(new_df)

df_new = pd.concat(new_subset_performance).reset_index(drop=True)




In [ ]:
def get_set_type(x: str) -> str:
    if x.startswith("class"):
        _id = "StratifiedShuffleSplit" if "StratifiedShuffleSplit" in x else "ShuffleSplit"
        if x.endswith("dev"):
            return f"{_id}_dev"
        return f"{_id}_test"
    if x.startswith("pareto"):
        _id = "pareto"
        if x.endswith("dev"):
            return f"{_id}_dev"
        return f"{_id}_test"
    return x

## Average seeds
Get data: performance on optimizer_id/task_id

In [ ]:
df_crit = {}
for set_id, gdf in df_new.groupby("set"):
    df_crit[set_id] = get_df_crit(gdf, perf_col="trial_value__cost_inc_norm")

In [ ]:
df_crit_points = {k: v[original_optimizers[scenario]].values for k, v in df_crit.items()}

## Calculate and Plot Discrepancy
Note: This is the WD discrepancy, not exactly the one used for the subselection.
It should be precise enough however.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import qmc

sns.set_palette("colorblind")
sns.set_style("whitegrid")

discprecancies = {}
for set_id, points in df_crit_points.items():
    # print(set_id, points.shape)
    discprecancies[set_id] = qmc.discrepancy(points, method="WD")
discrepancies = pd.Series(discprecancies)
discrepancies.index.name = "set"
discrepancies.name = "discrepancy"
discrepancies = discrepancies.reset_index()
discrepancies["set_type"] = discrepancies["set"].apply(get_set_type)
discrepancies


fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)
ax = sns.scatterplot(data=discrepancies, x="discrepancy", y="set_type", ax=ax)
# ax.set_yscale("log")
# ax.tick_params(axis='x', rotation=90)
ax.set_title("discrepancy")
ax.figure.savefig(f"../data/{scenario}_discrepancy.pdf", bbox_inches="tight")

## Calc Ranks + Count How Often Different
Calc ranks with statistical tests/autorank and count how often the ranking differs between the dev and test set.

In [ ]:
_df = df_new

In [ ]:
%%capture --no-display
from autorank._util import get_sorted_rank_groups
from carps.analysis.plot_ranking import calc_critical_difference

rank_results = {}
for (scenario, set_id), gdf in _df.groupby(by=["scenario", "set"]):
    perf_col: str = "trial_value__cost_inc_norm"
    identifier = f"{scenario}_{set_id}"
    result = calc_critical_difference(gdf, identifier=identifier, figsize=(8, 3), perf_col=perf_col, plot_diagram=False)
    sorted_ranks, names, groups = get_sorted_rank_groups(result, reverse=False)
    rank_results[(scenario, set_id)] = result

In [ ]:
R = []
for i, (k, v) in enumerate(rank_results.items()):
    d = pd.DataFrame({
        "scenario": k[0],
        "set": k[1],
        **v.rankdf["meanrank"]
    }, index=[i]
    ).melt(id_vars=["scenario", "set"], var_name="optimizer_id", value_name="meanrank")
    d["order"] = d.rank(method="max", numeric_only=True).astype(int)
    R.append(d)
    # break

df_rank = pd.concat(R).reset_index(drop=True)
df_rank = df_rank[df_rank["set"] != "full"]
df_rank

In [ ]:

is_different = {}
for scenario, gdf in df_rank.groupby(by="scenario"):
    set_origins = gdf["set"].apply(lambda x: "_".join(x.split("_")[:-2])).unique()
    for set_origin in set_origins:
        origs = original_optimizers[scenario]
        df_dev = gdf[gdf["set"] == f"{set_origin}_subset_dev"]
        df_test = gdf[gdf["set"] == f"{set_origin}_subset_test"]
        order_dev = []
        order_test = []
        for orig in origs:
            order_dev.append(df_dev[df_dev["optimizer_id"] == orig]["order"].values[0])
            order_test.append(df_test[df_test["optimizer_id"] == orig]["order"].values[0])
        _is_different = order_dev != order_test
        is_different[(scenario, set_origin)] = _is_different
is_different

In [ ]:
df_diff = pd.Series({k[1]: v for k, v in is_different.items()})
df_diff.index.name = "origin_detail"
df_diff.name = "is_different"
df_diff = df_diff.reset_index()
df_diff["is_different"] = df_diff["is_different"].astype(int)
def get_origin(x: str) -> str:
    if x.startswith("class"):
        return x.split("_")[1]
    if x.startswith("pareto"):
        return "pareto"
    return x
df_diff["origin"] = df_diff["origin_detail"].apply(get_origin)
df_diff

df_diff_reduced = df_diff.groupby(by="origin").agg({"is_different": "sum"}).reset_index()
n_total_splits = n_splits * len(seeds)
df_diff_reduced["fraction_is_different"] = df_diff_reduced["is_different"] / n_total_splits
df_diff_reduced.to_latex(f"../data/{scenario}_is_different.tex", index=False)
df_diff_reduced

## (Print some tables)

In [ ]:
import numpy as np
import pandas as pd

fn_template = "ranks_per_set_{scenario}_{set_origin}.csv"

decimal_places = 2

final_str = r"""
\begin{{table}}[h]
    \caption{{{caption}}}
    \label{{{label}}}
    \centering
    %\resizebox{{0.4\textwidth}}{{!}}{{
    {table_string}
    %}}
\end{{table}}
"""

float_format = lambda x: ("{:0." + str(decimal_places) + "f}").format(x) if not np.isnan(x) else "-"


for scenario, gdf in df_rank.groupby("scenario"):
    set_origins = gdf["set"].apply(lambda x: "_".join(x.split("_")[:-2])).unique()
    for set_origin in set_origins:
        fn = fn_template.format(scenario=scenario, set_origin=set_origin)


        sorter = gdf[gdf["set"] == set_origin + "_subset_dev"].sort_values("meanrank")["optimizer_id"].to_list()

        _gdf = gdf[gdf["set"].apply(lambda x: x.startswith(set_origin))].copy()
        R = _gdf.pivot_table(index="set", columns="optimizer_id", values="order").map(int)
        origs = original_optimizers[scenario]
        origs.sort(key=lambda x: sorter.index(x))
        cols = origs + [c for c in R.columns if c not in original_optimizers[scenario]]
        R = R[cols]


        MR = _gdf.pivot_table(index="set", columns="optimizer_id", values="meanrank").map(lambda x: f"{x:.2f}" if not isinstance(x, str) else x)
        MR = MR[cols]
        for i, ((_idx, row), (_idx2, row2)) in enumerate(zip(MR.iterrows(), R.iterrows(), strict=False)):
            for j in range(len(row)):
                row.iloc[j] = row.iloc[j] + f" ({int(row2.iloc[j])})"

        print(MR)
        # table_str = MR.to_latex(float_format=float_format, na_rep="-").strip()
        # caption = f"Mean Ranking for Scenario {scenario}"
        # label = f"tab:ranking_validation_{scenario}"
        # table_str = final_str.format(table_string=table_str, label=label, caption=caption)
        # table_str = table_str.replace("_", "\_")

        # with open(fn + ".tex", "w") as file:
        #     file.write(table_str)
        # print(table_str)